In [ ]:
import pandas as pd
import sys
sys.path.append("../../Muestreador")
from muestreador import MuestreoINE
m = MuestreoINE()

In [ ]:
personas = pd.read_spss("ENEI2022.sav")
hogares = pd.read_spss("ENEI2022hogares.sav")
hogares.head(3)

In [ ]:
# Estimar el promedio de personas por hogar por dominio de estudio
agrup_regiones = hogares.groupby("dominio")
media_personas_hogar = {region : 0 for region in hogares["dominio"].unique().tolist()}
for region in hogares["dominio"].unique().tolist():
    media_personas_hogar[region] = m.media_estratificado("factor", "P01B04", hogares[hogares["dominio"] == region])

In [ ]:
media_personas_hogar

In [ ]:
# Agregar columna de cantidad de personas por hogar con edad mayor o igual a 15 años
pea_hogares = personas[personas["P03A03"] >= 15].groupby(["dominio", "hogar_num"])["pea"].sum().fillna(0).reset_index(name="PEA hogar")
personas = pd.merge(personas, pea_hogares, how="outer", on=["dominio", "hogar_num"], suffixes=["","_x"])
personas["factor"] = personas["factor"].fillna(0)

In [ ]:
# Estimar promedio de personas en edad de trabajar por dominio de estudio
media_pea_hogar = {region : 0 for region in personas["dominio"].unique().tolist()}
for region in personas["dominio"].unique().tolist():
    media_pea_hogar[region] = m.media_estratificado("factor", "PEA hogar", personas[personas["dominio"] == region])

In [13]:
media_pea_hogar

{'Urbano Metropolitano': 1.9390919533473672,
 'Rural Nacional': 1.896876797873538,
 'Resto Urbano': 2.05796369071623}

In [ ]:
# Estimar porcentaje de personas susceptibles a ser observadas
r = {region : 0 for region in personas["dominio"].unique().tolist()}
for region in personas["dominio"].unique().tolist():
    r[region] = media_pea_hogar[region] / media_personas_hogar[region]

In [11]:
# Estimar total personas de 15 años o más en PET
personas["PET con peso"] = personas["factor"] * personas["PEA hogar"]
personas_unicos = personas.drop_duplicates(subset="hogar_num", keep="first")
personas_unicos.groupby("dominio")["PET con peso"].sum()

dominio
Resto Urbano            1878665.0
Rural Nacional          3871194.0
Urbano Metropolitano    1416011.0
Name: PET con peso, dtype: float64

In [12]:
# Estimar total personas de 15 años o más en PET
personas["PET con peso"] = personas["factor"] * personas["pea"]
personas.groupby("dominio")["PET con peso"].sum()

dominio
Resto Urbano            1934262.0
Rural Nacional          4135996.0
Urbano Metropolitano    1433558.0
Name: PET con peso, dtype: float64

In [ ]:
a.groupby("dominio")["PEA hogar"].sum()

In [ ]:
personas.groupby(["dominio", "hogar_num"])["PEA hogar"]